## This is step 2

Step 1. [Download history price and clean data](/notebooks/forex/hawkeye/1_download.ipynb)

**Step 2. Use monte carlo simulate each stock's movement for 10000 times**

Step 3. [Apply some heuristic rules to give stock recommendation](/notebooks/forex/hawkeye/3_recommendation.ipynb)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import io
import csv
import requests
import pandas as pd
from decimal import Decimal

base_path = '/opt/hawkeye'
def get_asx_df():
    asx_url = 'https://www.asx.com.au/asx/research/ASXListedCompanies.csv'
    asx_data = requests.get(asx_url).content
    asx_df = pd.read_csv(io.StringIO(asx_data.decode('utf-8')), skiprows=1)

    return asx_df


df = get_asx_df()

In [2]:

def download_csv(code, local_priori=False):
    path = f'{base_path}/price/{code}.csv'
    if local_priori and os.path.exists(path):
        df = pd.read_csv(path, index_col='date')
    else:
        df, meta_data = ts.get_daily_adjusted(symbol=f'{code}.AUS')
        df.to_csv(path)
    return df

In [3]:

def stock_monte_carlo(start_price, days, mu, sigma):
    ''' This function takes in starting stock price, days of simulation,mu,sigma, and returns simulated price array'''
    dt = 1 / days
    price = np.zeros(days)
    price[0] = start_price

    shock = np.zeros(days)
    drift = np.zeros(days)

    for x in range(1, days):
        shock[x] = np.random.normal(loc=mu * dt, scale=sigma * np.sqrt(dt))
        drift[x] = mu * dt
        price[x] = price[x - 1] + (price[x - 1] * (drift[x] + shock[x]))
    return price


def monte_carlo_simulations(start_price, days, mu, sigma, runs=10000):
    # Create an empty matrix to hold the end price data
    simulations = np.zeros(runs)

    # Set the print options of numpy to only display 0-5 points from an array to suppress output
    np.set_printoptions(threshold=5)

    for run in range(runs):
        # Set the simulation data point as the last stock price for that run
        simulations[run] = stock_monte_carlo(start_price, days, mu, sigma)[days - 1]

    return simulations


In [4]:

def process_stock(code, name):
    df = download_csv(code, True)

    df['return'] = df['5. adjusted close'].pct_change(1)
    df = df.dropna()
    volume_mean = df['6. volume'].mean()
    return_mean = df['return'].mean()
    return_sigma = df['return'].std()
    start_price = df['4. close'][-1]
    # print(start_price, volume_mean, return_mean, return_sigma)

    days = 30
    simulations = monte_carlo_simulations(start_price, days, return_mean, return_sigma)

    percent99 = np.percentile(simulations, 1)
    percent90 = np.percentile(simulations, 10)
    percent80 = np.percentile(simulations, 20)
    percent70 = np.percentile(simulations, 30)
    percent60 = np.percentile(simulations, 40)

    sim_mean = simulations.mean()
    var = start_price - percent99

    # print(percent99, percent90, percent80, percent70, percent60)
    # print(sim_mean, var)

    # define q as the 1% empirical qunatile, this basically means that 99% of the values should fall between here

    # plot the distribution of the end prices
    plt.hist(simulations, bins=200)

    # Using plt.figtext to fill in some additional information onto the plot

    # Starting Price
    plt.figtext(0.6, 0.8, s="Start price: $%.2f" % start_price)
    # Mean ending price
    plt.figtext(0.6, 0.7, "Mean final price: $%.2f" % sim_mean)
    plt.figtext(0.6, 0.6, "VaR(0.99): $%.2f" % (var,))
    plt.figtext(0.15, 0.6, "q(0.99): $%.2f" % percent99)
    plt.axvline(x=percent99, linewidth=1, color='r')
    plt.axvline(x=percent90, linewidth=1, color='r')
    plt.axvline(x=percent80, linewidth=1, color='r')
    plt.axvline(x=percent70, linewidth=1, color='r')
    plt.axvline(x=percent60, linewidth=1, color='r')
    plt.title(f"Final price distribution for {name} Stock after %s {days}", weight='bold')

    plt.savefig(f'/opt/hawkeye/pic/{code}.png', format='png')
    plt.clf()
    plt.cla()
    plt.close()

    # print(code, start_price, sim_mean, float(var))
    return df.iloc[-1].name, start_price, sim_mean, Decimal(sim_mean - start_price).quantize(
        Decimal('0.000000000000001')), \
           Decimal(var).quantize(Decimal('0.000000000000001')), \
           Decimal((var) / start_price * 100).quantize(
               Decimal('0.001')), percent99, percent90, percent80, percent70, percent60, volume_mean


In [ ]:
no_data = []
errors = []
result_path = f'{base_path}/result.csv'
with open(result_path, 'a') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([
        'code', 'last_date', 'start price', 'sim_mean', 'sim_diff', 'VaR 99%',
        'VaR 99% Percent', 'volume_mean', 'return_mean', 'return_sigma',
        'percent99', 'percent90', 'percent80', 'percent70', 'percent60'
    ])

for i in range(len(df)):
    code = df.iloc[i]['ASX code']
    name = df.iloc[i]['Company name']
    path = f'{base_path}/price/{code}.csv'

    if not os.path.exists(path):
        no_data.append(code)
        print(i, code, 'No data skipped.')
        continue

    try:
        result = process_stock(code, name)
    except Exception as ex:
        errors.append(code)
        print(f'{i}. {code} raise error: {ex}')
        continue
    with open(result_path, 'a') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow((code, ) + result)

    print(i, code)

print(f'No data: {no_data}')
print(f'Error: {errors}')

0 MOQ
1 ONT
2 14D
3 1ST
4 T3D
5 TGP
6 TOT
7 TDO
8 DDD
9 3PL
10 4DS
11 5GN
12 88E
13 8CO
14. 8IH raise error: scale < 0
15 8EC
16 8VI
17 9SP
18 NNW
19 ACB
20. AYI raise error: scale < 0
21 A2B
22 ABP
23 ABL
24 AEG
25 ABT
26 AJC
27 AKG
28 AX8
29 AX1
30. ACS raise error: scale < 0
31 ACQ
32 ACF
33 ACR
34 ACW
35 AIV
36 ADA
37 ADH
38 1AD
39 ADD
40 AAU
41 ABC
42 ADR
43 AHZ
44 ADY
45 ADT
46 ADJ
47 ANO
48 ABV
49 ASW
50 AV1
51 ADX
52 AER
53 AML
54 AEI
55 AIS
56 AEB No data skipped.
57 AFR
58. A1G raise error: scale < 0
59 AFT No data skipped.
60 AFP
61 APT
62 AGL
63 AGJ
64 AMN
65 AGR
66 AHL
67 A1C
68 APW
69 AGI
70 AIZ
71 AXP
72 AJL
73 AQG
74 AUQ
75 ACL
76 ALY
77 ALC
78 AL8
79 ARN
80 LEP
81 AJX
82 AQI
